# Fire Detection Using the MobileNetV2 Model

## Importing the libraries

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.__version__

'2.9.1'

## Preprocessing the Training and Validation Data Sets

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True) 
training_set = train_datagen.flow_from_directory(
        '/Users/aayushsubramaniam/Downloads/FIRE-SMOKE-DATASET/Train',
        target_size=(224, 224),
        batch_size=128,
        shuffle=True,
        class_mode='categorical')

validation_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True) 

validation_set = validation_datagen.flow_from_directory(
    '/Users/aayushsubramaniam/Downloads/FIRE-SMOKE-DATASET/Test',
    target_size=(224,224),
    batch_size=20,
    class_mode='categorical',
    shuffle=True)

Found 1800 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


## The MobileNetV2 Model

In [4]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Dropout
input_tensor = Input(shape=(224, 224, 3)) #Keras Input Tensor Object
base_model = MobileNetV2(input_tensor=input_tensor, weights='imagenet', include_top=False) # Returns a Keras Model Object
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(2048, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(2, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
  layer.trainable = False
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

history = model.fit(
training_set,
steps_per_epoch = 14,
epochs = 20,
validation_data = validation_set,
validation_steps = 14)

Epoch 1/20
14/14 [==============================] - 68s 5s/step - loss: 6.0107 - acc: 0.7356 - val_loss: 0.2925 - val_acc: 0.8750
Epoch 2/20
14/14 [==============================] - 56s 4s/step - loss: 0.3387 - acc: 0.8995
Epoch 3/20
14/14 [==============================] - 55s 4s/step - loss: 0.1449 - acc: 0.9420
Epoch 4/20
14/14 [==============================] - 56s 4s/step - loss: 0.2311 - acc: 0.9145
Epoch 5/20
14/14 [==============================] - 56s 4s/step - loss: 0.2028 - acc: 0.9270
Epoch 6/20
14/14 [==============================] - 56s 4s/step - loss: 0.0707 - acc: 0.9743
Epoch 7/20
14/14 [==============================] - 56s 4s/step - loss: 0.0578 - acc: 0.9803
Epoch 8/20
14/14 [==============================] - 55s 4s/step - loss: 0.2676 - acc: 0.9318
Epoch 9/20
14/14 [==============================] - 90s 7s/step - loss: 0.1325 - acc: 0.9659
Epoch 10/20
14/14 [==============================] - 57s 4s/step - loss: 0.0366 - acc: 0.9827
Epoch 11/20
14/14 [=============

In [5]:
for layer in model.layers[:249]:
  layer.trainable = False
for layer in model.layers[249:]:
  layer.trainable = True

In [6]:
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['acc'])
history = model.fit(
training_set,
steps_per_epoch = 14,
epochs = 10,
validation_data = validation_set,
validation_steps = 14)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/10
14/14 [==============================] - 39s 3s/step - loss: 0.0375 - acc: 0.9880 - val_loss: 0.2506 - val_acc: 0.9600
Epoch 2/10
14/14 [==============================] - 34s 2s/step - loss: 0.0255 - acc: 0.9877
Epoch 3/10
14/14 [==============================] - 32s 2s/step - loss: 0.0291 - acc: 0.9880
Epoch 4/10
14/14 [==============================] - 32s 2s/step - loss: 0.0327 - acc: 0.9868
Epoch 5/10
14/14 [==============================] - 32s 2s/step - loss: 0.0306 - acc: 0.9886
Epoch 6/10
14/14 [==============================] - 32s 2s/step - loss: 0.0387 - acc: 0.9886
Epoch 7/10
14/14 [==============================] - 32s 2s/step - loss: 0.0323 - acc: 0.9868
Epoch 8/10
14/14 [==============================] - 32s 2s/step - loss: 0.0299 - acc: 0.9892
Epoch 9/10
14/14 [==============================] - 32s 2s/step - loss: 0.0339 - acc: 0.9898
Epoch 10/10
14/14 [==============================] - 33s 2s/step - loss: 0.0335 - acc: 0.9862


Note: The Functions of Transfer Learning & Fine Tuning were performed

References:


1.   https://keras.io/api/applications/mobilenet/
2.   https://keras.io/guides/transfer_learning/#freezing-layers-understanding-the-trainable-attribute
3.   https://keras.io/api/applications/#usage-examples-for-image-classification-models


### Making a Single Prediction

In [36]:
import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('/Users/aayushsubramaniam/Downloads/picture.jpg', target_size = (224,224))

In [37]:
test_image = image.img_to_array(test_image) 
test_image = np.expand_dims(test_image, axis = 0) 
results = model.predict(test_image/255.0)
training_set.class_indices 

1/1 [==============================] - 0s 393ms/step


{'Fire': 0, 'Neutral': 1}

In [39]:
class_name = ['Fire','Neutral']

In [40]:
score = tf.nn.softmax(results[0])
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_name[np.argmax(score)], 100 * np.max(score))
)

This image most likely belongs to Fire with a 73.11 percent confidence.


In [17]:
print(prediction)

Fire


### Saving the model

In [19]:
new_model = tf.keras.models.load_model('FireDetection.pb')

new_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

 block_3_expand_relu (ReLU)     (None, 56, 56, 144)  0           ['block_3_expand_BN[0][0]']      
                                                                                                  
 block_3_pad (ZeroPadding2D)    (None, 57, 57, 144)  0           ['block_3_expand_relu[0][0]']    
                                                                                                  
 block_3_depthwise (DepthwiseCo  (None, 28, 28, 144)  1296       ['block_3_pad[0][0]']            
 nv2D)                                                                                            
                                                                                                  
 block_3_depthwise_BN (BatchNor  (None, 28, 28, 144)  576        ['block_3_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_3_d

 lization)                                                                                        
                                                                                                  
 block_7_expand (Conv2D)        (None, 14, 14, 384)  24576       ['block_6_project_BN[0][0]']     
                                                                                                  
 block_7_expand_BN (BatchNormal  (None, 14, 14, 384)  1536       ['block_7_expand[0][0]']         
 ization)                                                                                         
                                                                                                  
 block_7_expand_relu (ReLU)     (None, 14, 14, 384)  0           ['block_7_expand_BN[0][0]']      
                                                                                                  
 block_7_depthwise (DepthwiseCo  (None, 14, 14, 384)  3456       ['block_7_expand_relu[0][0]']    
 nv2D)    

                                                                                                  
 block_10_depthwise_relu (ReLU)  (None, 14, 14, 384)  0          ['block_10_depthwise_BN[0][0]']  
                                                                                                  
 block_10_project (Conv2D)      (None, 14, 14, 96)   36864       ['block_10_depthwise_relu[0][0]']
                                                                                                  
 block_10_project_BN (BatchNorm  (None, 14, 14, 96)  384         ['block_10_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_11_expand (Conv2D)       (None, 14, 14, 576)  55296       ['block_10_project_BN[0][0]']    
                                                                                                  
 block_11_

 block_14_depthwise (DepthwiseC  (None, 7, 7, 960)   8640        ['block_14_expand_relu[0][0]']   
 onv2D)                                                                                           
                                                                                                  
 block_14_depthwise_BN (BatchNo  (None, 7, 7, 960)   3840        ['block_14_depthwise[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 block_14_depthwise_relu (ReLU)  (None, 7, 7, 960)   0           ['block_14_depthwise_BN[0][0]']  
                                                                                                  
 block_14_project (Conv2D)      (None, 7, 7, 160)    153600      ['block_14_depthwise_relu[0][0]']
                                                                                                  
 block_14_

## Tensorflow - Lite

In [21]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
open('FireDetection.tflite','wb').write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/dq/th4jrf_53n14t3xrph6gtdjh0000gn/T/tmpxph9hkxs/assets


INFO:tensorflow:Assets written to: /var/folders/dq/th4jrf_53n14t3xrph6gtdjh0000gn/T/tmpxph9hkxs/assets


7234880